<h1 style="text-align:center">Blood Cell Dataset Classification</h1>
<h4 style="text-align:center">by Junhe Zhang</h4>

# Load Useful Libraries

In [1]:
import torch                                        # root package
from torch.utils.data import Dataset    # dataset representation and loading

In [2]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [4]:
# for reading and displaying images
from skimage.io import imread
import matplotlib.pyplot as plt
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
%matplotlib inline

# Load Cell Image Dataset

In [5]:
# load image data
import os
import numpy as np
import cv2

DATA_DIR = os.getcwd() + "/train/"
RESIZE_TO = 256

x, y = [], []
idx = 0
for path in [f for f in os.listdir(DATA_DIR) if f[-4:] == ".png"]:
    x.append(cv2.resize(cv2.imread(DATA_DIR + path), (RESIZE_TO, RESIZE_TO)))
    idx+=1
    with open(DATA_DIR + path[:-4] + ".txt", "r") as s:
        label = s.read()
    label = label.split('\n')
    y.append(label)


In [6]:
y

[['red blood cell', 'schizont'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'difficult', 'trophozoite', 'ring'],
 ['red blood cell', 'difficult'],
 ['red blood cell', 'trophozoite', 'ring'],
 ['red blood cell', 'ring', 'trophozoite', 'difficult'],
 ['red blood cell', 'schizont', 'difficult'],
 ['red blood cell', 'ring'],
 ['red blood cell', 'ring'],
 ['red blood cell', 'ring'],
 ['red blood cell', 'difficult', 'ring', 'trophozoite'],
 ['red blood cell', 'trophozoite', 'difficult'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'ring', 'leukocyte', 'trophozoite'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'trophozoite', 'ring'],
 ['red blood cell', 'trophozoite', 'gametocyte'],
 ['red blood cell', 'ring', 'difficult', 'trophozoite'],
 ['red blood cell', 'schizont', 'trophozoite'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'trophozoite'],
 ['red blood cell', 'ring', 

# Encode Multiple Labels

In [7]:
# Create MultiLabelBinarizer object
one_hot = MultiLabelBinarizer(["red blood cell", "difficult", "gametocyte", "trophozoite", "ring", "schizont", "leukocyte"])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=['red blood cell', 'difficult', 'gametocyte', 'trophozoite', 'ring', 'schizont', 'leukocyte'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [8]:
# One-hot encode data
y_encoded = one_hot.fit_transform(y)

In [9]:
y_encoded[0]

array([1, 0, 0, 0, 0, 1, 0])

In [10]:
# list to ndarray
x_np = np.array(x)
y_np = np.array(y_encoded)

In [11]:
x_row = len(x_np)
x_row

929

In [12]:
x_np.shape

(929, 256, 256, 3)

# Normalized Data
Normalized data to get more efficient training process

In [13]:
# normalize 4d dataset
x_min = x_np.min(axis=(1, 2), keepdims=True)
x_max = x_np.max(axis=(1, 2), keepdims=True)
x_norm = (x_np - x_min)/(x_max-x_min)
x_norm

array([[[[0.97916667, 0.99215686, 0.99190283],
         [0.99166667, 0.95686275, 0.94736842],
         [0.9       , 0.60392157, 0.67611336],
         ...,
         [0.99166667, 0.99215686, 0.99190283],
         [0.99166667, 0.99215686, 0.99190283],
         [0.99166667, 0.99215686, 0.99190283]],

        [[0.99166667, 0.98431373, 0.99190283],
         [0.95416667, 0.8627451 , 0.87449393],
         [0.77916667, 0.54509804, 0.60728745],
         ...,
         [0.99166667, 0.99215686, 0.99190283],
         [0.99166667, 0.99215686, 0.99190283],
         [0.99166667, 0.99215686, 0.99190283]],

        [[0.99166667, 0.99215686, 0.98380567],
         [0.97083333, 0.82352941, 0.8340081 ],
         [0.7875    , 0.56862745, 0.61133603],
         ...,
         [0.99166667, 0.99215686, 0.99190283],
         [0.99166667, 0.99215686, 0.99190283],
         [0.99166667, 0.99215686, 0.99190283]],

        ...,

        [[0.9875    , 0.98823529, 0.98785425],
         [0.9875    , 0.98823529, 0.98785425]

In [14]:
x_norm.shape

(929, 256, 256, 3)

In [15]:
x_np = x_norm.copy()

In [18]:
x_scaled = x_np.copy()
x_scaled[0]

array([[[0.97916667, 0.99215686, 0.99190283],
        [0.99166667, 0.95686275, 0.94736842],
        [0.9       , 0.60392157, 0.67611336],
        ...,
        [0.99166667, 0.99215686, 0.99190283],
        [0.99166667, 0.99215686, 0.99190283],
        [0.99166667, 0.99215686, 0.99190283]],

       [[0.99166667, 0.98431373, 0.99190283],
        [0.95416667, 0.8627451 , 0.87449393],
        [0.77916667, 0.54509804, 0.60728745],
        ...,
        [0.99166667, 0.99215686, 0.99190283],
        [0.99166667, 0.99215686, 0.99190283],
        [0.99166667, 0.99215686, 0.99190283]],

       [[0.99166667, 0.99215686, 0.98380567],
        [0.97083333, 0.82352941, 0.8340081 ],
        [0.7875    , 0.56862745, 0.61133603],
        ...,
        [0.99166667, 0.99215686, 0.99190283],
        [0.99166667, 0.99215686, 0.99190283],
        [0.99166667, 0.99215686, 0.99190283]],

       ...,

       [[0.9875    , 0.98823529, 0.98785425],
        [0.9875    , 0.98823529, 0.98785425],
        [0.9875    , 0

In [19]:
# split into train, valid, test
from sklearn.model_selection import train_test_split

In [20]:
X_train_np, X_test_np, y_train_np, y_test_np = train_test_split(x_scaled, y_np, test_size=0.25)

In [21]:
X_train_np.shape

(696, 256, 256, 3)

# Setup GPU
Activate GPU if available

In [22]:
# setup GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.set_device(torch.device('cuda:0'))
torch.backends.cudnn.benchmark = False

cuda:0


# Convert n-D Array to tensor for GPU

In [23]:
# ndarray to tensor cuda
X_train_tensor = torch.from_numpy(X_train_np).float().to(device)
X_test_tensor = torch.from_numpy(X_test_np).float().to(device)

In [24]:
X_test_tensor.shape

torch.Size([233, 256, 256, 3])

In [25]:
y_train_tensor = torch.from_numpy(y_train_np).float().to(device)
y_test_tensor = torch.from_numpy(y_test_np).float().to(device)


In [26]:
X_train_tensor.shape

torch.Size([696, 256, 256, 3])

In [27]:
X_test_tensor.shape

torch.Size([233, 256, 256, 3])

In [28]:
(y_train_tensor[0], y_test_tensor)

(tensor([1., 0., 1., 0., 0., 0., 0.], device='cuda:0'),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 0., 1., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'))

# Building model

In [33]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [34]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.ConvLayer1 = nn.Sequential(
        nn.Conv2d(3, 64, 3), # 3, 256, 256
        nn.BatchNorm2d(64),
        nn.MaxPool2d(2), # op: 16, 127, 127
        nn.ReLU(inplace=True), # op: 64, 127, 127
        )
        self.ConvLayer2 = nn.Sequential(
        nn.Conv2d(64, 128, 3), # 64, 127, 127
        nn.BatchNorm2d(128),
        nn.MaxPool2d(2), #op: 128, 63, 63
        nn.ReLU(inplace=True) # op: 128, 63, 63
        )
        self.ConvLayer3 = nn.Sequential(
        nn.Conv2d(128, 256, 3), # 128, 63, 63
        nn.BatchNorm2d(256),
        nn.MaxPool2d(2), #op: 256, 30, 30
        nn.ReLU(inplace=True), #op: 256, 30, 30
        nn.Dropout(0.2)
        )
        self.ConvLayer4 = nn.Sequential(
        nn.Conv2d(256, 512, 3), # 256, 30, 30
        nn.BatchNorm2d(512),
        nn.MaxPool2d(2), #op: 512, 14, 14
        nn.ReLU(inplace=True), #op: 512, 14, 14
        nn.Dropout(0.2)
        )
        self.Linear1 = nn.Linear(512 * 14 * 14, 1024)
        self.Linear2 = nn.Linear(1024, 256)
        self.Linear3 = nn.Linear(256, 7)
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0), -1)
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return x

In [38]:
def check_cuda():
    _cuda = False
    if torch.cuda.is_available():
        _cuda = True
    return _cuda
is_cuda = check_cuda()
# model = MultiClassifier()

In [39]:
from torch.autograd import Variable
# change input shape to (B,H,W,C) for con2d
inputs, labels = Variable(X_train_tensor), Variable(y_train_tensor)
inputs = inputs.transpose(1,3)

In [40]:
inputs.shape

torch.Size([696, 3, 256, 256])

# Setup Loss function and Optimizer

In [42]:
model_dir = os.getcwd() + "/model/"
model = MultiClassifier()
# defining the optimizer
# optimizer = Adam(model.parameters(), lr=0.02)
# defining the loss function
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.0001, momentum = 0.9, weight_decay = 1e-5)
batch_size = 6
# model = nn.Linear(X_train_tensor.shape[1], 7) # predict logits for 7 classes
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    print('cuda')
else:
    print('cpu')

cuda


In [44]:
# load training model if exist
model.load_state_dict(torch.load("model_jzhang46.pt"))

<All keys matched successfully>

In [ ]:
# set number of time to train
# each training time take a long time
train_time = 10

# Start Training

In [50]:
# Start Training
for i in range(train_time):
    print('train time: {}'.format(i))
    for epoch in range(20):
        model.train()
        acc = 0
        for batch in range(len(inputs)//batch_size):
            idx = slice(batch*batch_size, (batch+1)*batch_size)
            optimizer.zero_grad()
            output = torch.sigmoid(model(inputs[idx]))
            loss = criterion(output, labels[idx])
            loss.backward()
            optimizer.step()
        print('Loss: {:.6f}'.format(loss.item()))
    torch.save(model.state_dict(), "model_jzhang46.pt")
    torch.cuda.empty_cache()

train time: 0
Loss: 0.000522
Loss: 0.000463
Loss: 0.000832
Loss: 0.000416
Loss: 0.000397
Loss: 0.000384
Loss: 0.000424
Loss: 0.000438
Loss: 0.000429
Loss: 0.000413
Loss: 0.000593
Loss: 0.000525
Loss: 0.000281
Loss: 0.000363
Loss: 0.000338
Loss: 0.000249
Loss: 0.000379
Loss: 0.000454
Loss: 0.000499
Loss: 0.000426
train time: 1
Loss: 0.000626
Loss: 0.000542
Loss: 0.000438
Loss: 0.000480
Loss: 0.000391
Loss: 0.000585
Loss: 0.000352
Loss: 0.000329
Loss: 0.000445
Loss: 0.000497
Loss: 0.000526
Loss: 0.000420
Loss: 0.000390
Loss: 0.000485
Loss: 0.000365
Loss: 0.000406
Loss: 0.000448
Loss: 0.000367
Loss: 0.000236
Loss: 0.000454
train time: 2
Loss: 0.000332
Loss: 0.000421
Loss: 0.000427
Loss: 0.000563
Loss: 0.000358
Loss: 0.000465
Loss: 0.000261
Loss: 0.000396
Loss: 0.000487
Loss: 0.000324
Loss: 0.000388
Loss: 0.000267
Loss: 0.000402
Loss: 0.000431
Loss: 0.000360
Loss: 0.000439
Loss: 0.000316
Loss: 0.000455
Loss: 0.000762
Loss: 0.000360
train time: 3
Loss: 0.000345
Loss: 0.000423
Loss: 0.000366

# Save Trained model for futher training

In [47]:
torch.save(model.state_dict(), "model_jzhang46.pt")

# Load Trained model

In [ ]:
model.load_state_dict(torch.load("model_jzhang46.pt"))

In [63]:
labels[3]

tensor([1., 0., 0., 0., 1., 0., 0.], device='cuda:0')

In [66]:
torch.sigmoid(model(inputs[slice(2, 3)]))

tensor([[9.9998e-01, 1.7081e-01, 4.7955e-04, 1.6860e-02, 6.6275e-01, 8.0887e-02,
         4.2859e-02]], device='cuda:0', grad_fn=<SigmoidBackward>)

# Evaluate the trained model on test data

In [52]:
model.eval()

MultiClassifier(
  (ConvLayer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU(inplace=True)
  )
  (ConvLayer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU(inplace=True)
  )
  (ConvLayer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU(inplace=True)
    (4): Dropout(p=0.2, inplace=False)
  )
  (ConvLayer4): Sequential(
    (0): Conv2d(256, 512, kernel_siz